In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import glob

hives_ids = ["smrpiclient7", "smrpiclient6", "smrpiclient3"]

In [ ]:
import matplotlib.pyplot as plt

%matplotlib widget

def plot_spectrogram(frequency, time_x, spectrocgram, title):
    fig = plt.figure(figsize=(6,4))
    plt.title(title)
    plt.pcolormesh(time_x, frequency, spectrocgram)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class View(nn.Module):
    """ Function for nn.Sequentional to reshape data """
    def __init__(self, shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)

def conv2d_block(in_f, out_f, *args, **kwargs):
    """ Function for building convolutional block

        Attributes
            in_f - number of input features
            out_f - number of output features
    """
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU(),
        nn.Dropout2d(p=0.2)
    )

def conv2d_transpose_block(in_f, out_f, *args, **kwargs):
    """ Function for building transpose convolutional block
        
        Attributes
            in_f - number of input features
            out_f - number of output features
    """
    return nn.Sequential(
        nn.ConvTranspose2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU(),
        nn.Dropout2d(p=0.2)
    )

######################################
#                                    #
#   Main convolutional autoencoder   #
#                                    #
######################################
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        
        ## encoder layers ##
        self.encoder = nn.Sequential(
            # [1x256x64] => [64x256x64]
            conv2d_block(1, 64, kernel_size=3, padding=1),
            # [64x256x64] => [64x128x32]
            nn.MaxPool2d(2, 2),
            # [64x128x32] => [32x128x32]
            conv2d_block(64, 32, kernel_size=3, padding=1),
            # [32x128x32] => [32x64x16]
            nn.MaxPool2d(2, 2),
            # [32x64x16] => [16x64x16]
            conv2d_block(32, 16, kernel_size=3, padding=1),
            # [16x64x16] => [16x32x8]
            nn.MaxPool2d(2, 2),
            # [16x32x8] => [4x32x8]
            conv2d_block(16, 4, kernel_size=3, padding=1),
            # [4x32x8] => [4x16x4]
            nn.MaxPool2d(2, 2),
            # [4x16x4] => [1x256]
            nn.Flatten(),
            # [1x256] => [1x64]
            nn.Linear(256, 64),
            nn.ReLU()
        )
        
        ## decoder layers ##
        self.decoder = nn.Sequential(
            # [1x64] => [1x256]
            nn.Linear(64, 256),
            nn.ReLU(),
            # [1x256] => [4x16x4]
            View([-1, 4, 16, 4]),
            # [4x16x4] => [16x32x8]
            conv2d_transpose_block(4, 16, kernel_size=2, stride=2),
            # [16x32x8] => [32x64x16]
            conv2d_transpose_block(16, 32, kernel_size=2, stride=2),
            # [32x64x16] => [64x128x32]
            conv2d_transpose_block(32, 64, kernel_size=2, stride=2),
            # [64x128x32] => [1x256x64]
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2),
            nn.Sigmoid()
        )


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        
        return x

In [ ]:
import torch
from torch import nn

class autoencoder_basic(nn.Module):
    def __init__(self):
        super(autoencoder_basic, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1499, 512),
            nn.SELU(True),
            nn.Linear(512, 128),
            nn.SELU(True),
            nn.Linear(128, 64),
            nn.SELU(True))
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.SELU(True),
            nn.Linear(128, 512),
            nn.SELU(True),
            nn.Linear(128, 1499),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


## Load train data samples

In [3]:
import struct
import math
import numpy as np
import librosa
import librosa.display

from tqdm import tqdm
from datetime import datetime
from scipy.io import wavfile
from scipy import signal
from scipy.fftpack import fft

import matplotlib.pyplot as plt
%matplotlib widget

sound_time_ms = 2000
# ~93 ms for fft window
nfft = 4096
# ~34% overlapping
hop_len = (nfft//3) + 30
# This can be manipulated to adjust number of bins for conv layer
fmax = 2750

hives_data = []
max_to_norm = 0
for idx, hive_id in enumerate(hives_ids):
    sound_files = [f for f in glob.glob(f"..\\measurements\\smartulav2\\{hive_id}_*\\*.wav")]
    print(f"Sound data preparation for hive: {hive_id} which has {len(sound_files)} recordings...", end=' ', flush=True)
    for file in tqdm(sound_files):
        sample_rate, sound_samples = wavfile.read(file)
        sound_samples = sound_samples.T[0]/(2.0**31)
    
        spectrogram = librosa.core.stft(sound_samples, n_fft=nfft, hop_length=hop_len)
        spectrogram_magnitude = np.abs(spectrogram)
        spectrogram_phase = np.angle(spectrogram)
        spectrogram_db = librosa.amplitude_to_db(spectrogram_magnitude, ref=np.max)
        frequencies = librosa.fft_frequencies(sr=sample_rate, n_fft=nfft)
        times = (np.arange(0, spectrogram_magnitude.shape[1])*hop_len)/sample_rate
        
        freq_slice = np.where((frequencies < fmax))
        frequencies = frequencies[freq_slice]
        spectrogram_db = spectrogram_db[freq_slice, :][0]    
    
        filename = file.rsplit('\\', 1)[-1]
        timestamp = filename[filename.index('-')+1:].rsplit(".wav")[0]
        datetime = datetime.strptime(timestamp, '%Y-%m-%dT%H-%M-%S')
        hives_data.append([datetime, hive_id, sound_samples, [frequencies, times, spectrogram_db]])
    print(" done.")

print(f"Got {len(hives_data)} sound samples")

Sound data preparation for hive: smrpiclient7 which has 3367 recordings... 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3367/3367 [00:57<00:00, 58.37it/s]


 done.
Sound data preparation for hive: smrpiclient6 which has 3172 recordings... 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3172/3172 [00:53<00:00, 58.81it/s]

 done.
Sound data preparation for hive: smrpiclient3 which has 602 recordings... 


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 602/602 [00:10<00:00, 58.05it/s]

 done.
Got 7141 sound samples


## Prepare data for autoencoder train.

In [ ]:
import random 

random_idx = random.randint(0, len(hives_data) - 1)

plot_spectrogram(hives_data[random_idx][3][0],
                 hives_data[random_idx][3][1],
                 hives_data[random_idx][3][2],
                 f"hive: {hives_data[random_idx][1]}, time: {hives_data[random_idx][0]}, idx: {random_idx}") 

## Train basic AE

In [ ]:
from scipy import signal as sig
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
from torch.utils import data as tdata

# Divide data to test, validation and train
train_stop_idx = int(pd_ae_data.shape[0]*90/100)

pd_ae_data_train = pd_ae_data[:train_stop_idx]
pd_ae_data_test = pd_ae_data[train_stop_idx:]

print(f'Train data size: {pd_ae_data_train.shape[0]}')
print(f'Test data size: {pd_ae_data_test.shape[0]}')

tensor_train = torch.Tensor(pd_ae_data_train['periodogram'].values.tolist())
tensor_test = torch.Tensor(pd_ae_data_test['periodogram'].values.tolist())

train_dataset = tdata.TensorDataset(tensor_train)
test_dataset = tdata.TensorDataset(tensor_test)

dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch.autograd import Variable
from torchvision import transforms
import torch.nn.functional as F

num_epochs = 1000
learning_rate = 1e-3

model = autoencoder_basic().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    loss = 0
    for data in dataloader:
        periodogram = data[0].to(device)
        # ===================forward=====================
        output = model(periodogram)
        #train_loss = criterion(output, periodogram)
        train_loss = F.binary_cross_entropy(output, periodogram)
        # ===================backward====================
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        loss += train_loss.item()
    loss = loss / len(dataloader)
    if (epoch+1) % (num_epochs/10) == 0:
        print(f'epoch {epoch + 1}/{num_epochs}, loss:{loss}')

In [ ]:
import matplotlib.pyplot as plt

counter = 0
with torch.no_grad():
    loss_test = 0
    for data in dataloader_test:
        periodograms_test = data[0].to(device)
        output = model(periodograms_test)
        for idx, i in enumerate(output):
            #loss_test += nn.MSELoss()(periodograms_test[idx], i)
            loss_test += F.binary_cross_entropy(periodograms_test[idx], i)

loss_test = loss_test/len(pd_ae_data_test)
print(f'Final test loss: {loss_test}')


## Train CONV AE

In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

sc = StandardScaler()
mm = MinMaxScaler()

spectrogram_ae_data = [hive_data[3][2] for hive_data in hives_data]
standarized_ae_data = [sc.fit_transform(spec.T).T for spec in spectrogram_ae_data]
scaled_ae_data = [mm.fit_transform(stan.T).T for stan in standarized_ae_data]

print(f"Got dataset of size: {len(scaled_ae_data)}")

Got dataset of size: 7141


In [5]:
import torch 
from torch.utils import data as tdata

train_data_size = len(scaled_ae_data)*80//100
test_data_size = (len(scaled_ae_data) - train_data_size) // 50
val_data_size = len(scaled_ae_data) - train_data_size - test_data_size

dataset_tensor = torch.Tensor(scaled_ae_data)
print(f"Dataset shape: {dataset_tensor.shape}")
print(f"Train set size: {train_data_size}")
print(f"Test set size: {test_data_size}")
print(f"Validation set size: {val_data_size}")

# add one extra dimension as it is required for conv layer
dataset_tensor = dataset_tensor[:, None, :, :] 
dataset = tdata.TensorDataset(dataset_tensor)
train_set, test_set, val_set = torch.utils.data.random_split(dataset, [train_data_size, test_data_size, val_data_size])

dataloader_train = tdata.DataLoader(train_set, batch_size=32, shuffle=True)
dataloader_test = tdata.DataLoader(test_set, batch_size=32, shuffle=True)
dataloader_val = tdata.DataLoader(val_set, batch_size=32, shuffle=True)

Dataset shape: torch.Size([7141, 256, 64])
Train set size: 5712
Test set size: 28
Validation set size: 1401


In [7]:
%matplotlib widget

import sys
import matplotlib.pyplot as plt

num_epochs = 50
learning_rate = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# del modelConvAE

modelConvAE = ConvAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(modelConvAE.parameters(), lr=learning_rate, weight_decay=1e-6)

# monitor training loss per batch
train_loss = []
# monitor validation loss per batch
val_loss = []
# save avg train losses for early stopping visualization
avg_train_loss = []
# save avg train losses for early stopping visualization
avg_val_loss = [] 
# patience when stop training
patience = 10
# counter for patience in early sotpping
patience_counter = 0
# best validation score
best_val_loss = -1
# model checkpoint filename
checkpoint_filename = 'checkpoint.pth'
# early stopping epoch
win_epoch = 0
    
for epoch in range(1, num_epochs+1):    
    ###################
    # train the model #
    ###################
    modelConvAE.train()
    for data in dataloader_train:
        # transfer data to device
        periodogram = data[0].to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass
        outputs = modelConvAE(periodogram)
        # calculate the loss
        loss = criterion(outputs, periodogram)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss.append(float(loss.item()))
        
    ###################
    # val the model   #
    ###################
    modelConvAE.eval()
    for val_data in dataloader_val:
        # transfer data to device
        periodogram = data[0].to(device)
        # forward pass
        outputs = modelConvAE(periodogram)
        # calculate the loss
        loss = criterion(outputs, periodogram)
        # update running val loss
        val_loss.append(float(loss.item()))
    
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = np.average(train_loss)
    val_loss = np.average(val_loss)
    avg_train_loss.append(train_loss)
    avg_val_loss.append(val_loss)
    
    epoch_len = len(str(num_epochs))
    # print avg training statistics 
    print(f'[{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}] train_loss: {train_loss:.5f} valid_loss: {val_loss:.5f}', end=' ', flush=True)
    
    if val_loss < best_val_loss or best_val_loss == -1:
        # new checkpoint
        print("checkpoint!")
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(modelConvAE.state_dict(), checkpoint_filename)
        win_epoch = epoch
    elif patience_counter >= patience:
        print("early stopping.")
        print(f"=> loading checkpoint {checkpoint_filename}")
        device = torch.device("cuda")
        modelConvAE.load_state_dict(torch.load(checkpoint_filename))
        break
    else:
        print(".")
        patience_counter = patience_counter + 1
        
    # clear batch losses
    train_loss = []
    val_loss = []

fig = plt.figure(figsize=(10,5))
plt.plot(np.arange(1, epoch + 1), avg_train_loss, 'r', label="train loss")
plt.plot(np.arange(1, epoch + 1), avg_val_loss, 'b', label="validation loss")
plt.axvline(win_epoch, linestyle='--', color='g',label='Early Stopping Checkpoint')
plt.legend(loc='best')
plt.grid(True)
plt.show()

[ 1/50] train_loss: 0.07586 valid_loss: 0.05323 checkpoint!
[ 2/50] train_loss: 0.04580 valid_loss: 0.04042 checkpoint!
[ 3/50] train_loss: 0.04318 valid_loss: 0.04921 .
[ 4/50] train_loss: 0.04239 valid_loss: 0.04637 .
[ 5/50] train_loss: 0.04156 valid_loss: 0.03884 checkpoint!
[ 6/50] train_loss: 0.04093 valid_loss: 0.03479 checkpoint!
[ 7/50] train_loss: 0.04054 valid_loss: 0.03844 .
[ 8/50] train_loss: 0.04024 valid_loss: 0.03842 .
[ 9/50] train_loss: 0.03985 valid_loss: 0.02924 checkpoint!
[10/50] train_loss: 0.03979 valid_loss: 0.04215 .
[11/50] train_loss: 0.03950 valid_loss: 0.03861 .
[12/50] train_loss: 0.03946 valid_loss: 0.04066 .
[13/50] train_loss: 0.03930 valid_loss: 0.03680 .
[14/50] train_loss: 0.03923 valid_loss: 0.04122 .
[15/50] train_loss: 0.03909 valid_loss: 0.04223 .
[16/50] train_loss: 0.03896 valid_loss: 0.03667 .
[17/50] train_loss: 0.03891 valid_loss: 0.04697 .
[18/50] train_loss: 0.03884 valid_loss: 0.03654 .
[19/50] train_loss: 0.03879 valid_loss: 0.03980 .


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
import matplotlib.pyplot as plt

counter = 0
with torch.no_grad():
    loss_test = 0
    for data in dataloader_test:
        periodograms_test = data[0].to(device)
        output = modelConvAE(periodograms_test)
        for idx, output_elem in enumerate(output):
            loss_test += criterion(periodograms_test[idx], output_elem)

loss_test = loss_test/len(scaled_ae_data)
print(f'Final test loss: {loss_test}')

Final test loss: 0.00016501914069522172


# Plot and visualize (MFCC vs AE)

In [10]:
def conv2d_encode(model, data_intput):
    """ Function for encoding data and returning encoded """
    dataset_tensor = torch.Tensor(data_intput)
    dataset_tensor = dataset_tensor[:, None, :, :]
    dataset_tensor = tdata.TensorDataset(dataset_tensor)
    dataset = tdata.DataLoader(dataset_tensor, batch_size=32, shuffle=True)
    encoded_data = []
    
    model.eval()
    with torch.no_grad():
        for data in dataset:
            periodograms = data[0].to(device)
            output = modelConvAE.encoder(periodograms).cpu().numpy().squeeze()
            encoded_data.extend(output)
    
    return encoded_data
            
        

In [11]:
encoded_data = conv2d_encode(modelConvAE, scaled_ae_data)

In [13]:
%matplotlib widget
import matplotlib.pyplot as plt
import random

idx = random.randint(0, len(hives_data) - 1)
with torch.no_grad():
    fig, axs = plt.subplots(2, 1)
    frequencies = librosa.fft_frequencies(sr=sample_rate, n_fft=nfft)
    freq_slice = np.where((frequencies < fmax))
    frequencies = frequencies[freq_slice]
    times = (np.arange(0, spectrogram_magnitude.shape[1])*hop_len)/sample_rate    
    
    elem = scaled_ae_data[idx]
    elem = elem[None, None,: ,:]
    elem = torch.Tensor(elem)

    axs[0].pcolormesh(times, frequencies, scaled_ae_data[idx])
    axs[1].pcolormesh(times, frequencies, modelConvAE(elem.to(device)).cpu().numpy().squeeze())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-13-11d830e735aa>:17: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  axs[0].pcolormesh(times, frequencies, scaled_ae_data[idx])
<ipython-input-13-11d830e735aa>:18: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  axs[1].pcolormesh(times, frequencies, modelConvAE(elem.to(device)).cpu().numpy().squeeze())


# Dimension reduction - now we perform t-SNE and PCA to visualize the data

In [ ]:
from sklearn.manifold import TSNE

reduced_ae_tsne = TSNE(n_components=2, perplexity=100, learning_rate=500, verbose=1).fit_transform(output)
reduced_mfcc_tsne = TSNE(n_components=2, perplexity=100, learning_rate=500, verbose=1).fit_transform(pd_ae_data['mfcc'].values.tolist())

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

mfccs_standarized = StandardScaler().fit_transform(pd_ae_data['mfcc'].values.tolist())
ae_standarized = StandardScaler().fit_transform(output)
reduced_ae_pca = PCA(n_components = 2).fit_transform(ae_standarized)
reduced_mfcc_pca = PCA(n_components = 2).fit_transform(mfccs_standarized)

In [ ]:
import matplotlib.pyplot as plt

pca = PCA().fit(ae_standarized)

plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import fft, fftfreq

RECORD_TIME = 2
SAMPLE_RATE = 44100

data = hives_data[-10][2][:, 0]/(2.0**31)
datetime = hives_data[-1][0]
fft_data = abs(fft(data))
freqs = fftfreq(int(len(fft_data)/2), 1/SAMPLE_RATE)

fig, axs = plt.subplots(2)
fig.tight_layout(pad=3.0)
axs[0].set_title(f"Sound recording at {datetime} ({RECORD_TIME}s)")
axs[0].grid()
axs[0].set_xlabel('Time [sec]')
axs[0].plot(np.linspace(0, 2, len(data)), data)

axs[1].set_title("Periodogram")
axs[1].set_xticks(np.arange(0, (freqs.size/2), step=100))
axs[1].set_xticklabels(np.arange(0, (freqs.size/2), step=100, dtype=int), rotation=45)
axs[1].grid()
axs[1].set_xlabel('Frequency [Hz]')
axs[1].plot(freqs[1:1500], fft_data[1:1500], 'r')